In [1]:
import requests


url = "https://en.wikipedia.org/wiki/List_of_weather_records"

response = requests.get(url)

In [3]:
print("status code", response.status_code)

status code 200


In [4]:
response.text[:1000]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of weather records - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-dis

In [5]:
from bs4 import BeautifulSoup 
# Create soup based on html code (stored in the attribute text)
# of the response object
soup = BeautifulSoup(response.text)

In [7]:
# tag is table
# filter onto classes = "wikitable sortable"
table = soup.find("table", class_="wikitable sortable")

In [8]:
# List of all  `tr` tags 
# (except the first one as it is the header)
rows = table.find_all("tr")[1:]

In [21]:
row_example = rows[12]

tds = row_example.find_all("td")
href = tds[2].find("a").get("href")
color_code = tds[1].get("style").split(";")[0].replace("background: ", "")
temperature_str = tds[1].text

'50.4\xa0°C (122.7\xa0°F)\n'

In [26]:
raw_data = []

for row in rows:

    # All cells
    tds = row.find_all("td")

    # Temperature
    temperature_text = tds[1].text

    # Style to get background
    style = tds[1].get("style")

    # Color (background)
    color_code = style.split(";")[0].replace("background: ", "")

    # Url page of location
    a = tds[2].find("a")
    
    
    if a is not None:
        url_suffix = a.get("href")
    else:
        url_suffix = None
        
    raw_data.append(
        {
            "temperature_text":temperature_text,
            "color_code":color_code,
            "url_suffix":url_suffix
        }
    )

In [29]:
import pandas as pd
df = pd.DataFrame(raw_data)

In [30]:
df

,temperature_text,color_code,url_suffix
0,51.3 °C (124.3 °F)\n,#770000,/wiki/Ouargla
1,44.0 °C (111.2 °F)\n,#DC0000,"/wiki/Maun,_Botswana"
2,47.2 °C (117.0 °F)\n,#B00000,"/wiki/Dori,_Burkina_Faso"
3,48.0 °C (118.4 °F)\n,#A50000,/wiki/Faya-Largeau
4,36.0 °C (96.8 °F)\n,#FF2500,/wiki/Prince_Said_Ibrahim_International_Airport
...,...,...,...
141,38.0 °C (100.4 °F)\n,#FF1800,/wiki/Saint-Laurent-du-Maroni
142,46.2 °C (115.2 °F)\n,#BE0000,/w/index.php?title=Las_Palmas_(Paraguay)&actio...
143,41.6 °C (106.9 °F)\n,#FD0000,/wiki/I%C3%B1apari
144,44.0 °C (111.2 °F)\n,#DC0000,/wiki/Paysand%C3%BA


In [31]:
url_suffixes = df.url_suffix.tolist()

url_suffixes_page_exist = []

for url_suffix in  url_suffixes:
    if url_suffix is None:
        url_suffixes_page_exist.append(None)
    elif url_suffix.startswith("/wiki"):
        url_suffixes_page_exist.append(url_suffix)
    else:
        url_suffixes_page_exist.append(None)

In [33]:
import time

url_prefix = "https://en.wikipedia.org"


coordinates = []

for url_suffix in url_suffixes_page_exist:
    print(url_suffix)
    try:
        url_location = url_prefix + url_suffix
        response_location = requests.get(url_location)
        coords = BeautifulSoup(response_location.text).find(id="coordinates")
        if coords is not None:
            lat, long = coords.find("span", class_="latitude").text,coords.find("span", class_="longitude").text
        else:
            lat, long = None, None
            
        
    except:
        lat, long = None, None
    
    coordinates.append((lat, long))
    time.sleep(1)

/wiki/Ouargla
/wiki/Maun,_Botswana
/wiki/Dori,_Burkina_Faso
/wiki/Faya-Largeau
/wiki/Prince_Said_Ibrahim_International_Airport
/wiki/Aswan
/wiki/Lavumisa
None
/wiki/Navrongo
/wiki/Koundara
None
/wiki/Ejeda
/wiki/Agadir
/wiki/Diffa
/wiki/Yola,_Nigeria
/wiki/Le_Port,_R%C3%A9union
None
/wiki/Dongola
/wiki/Kebili
/wiki/Semara
/wiki/Mfuwe
/wiki/Signy_Research_Station
/wiki/Sukhumi,_Abkhazia
/wiki/Farah,_Afghanistan
/wiki/Rajshahi
/wiki/Phuentsholing
/wiki/Preah_Vihear_Province
None
/wiki/Sheung_Shui
/wiki/Phalodi
/wiki/Banjarbaru
/wiki/Ahvaz
/wiki/Basra
/wiki/Tirat_Zvi
/wiki/Kumagaya,_Saitama
None
/wiki/Turkistan_(city)
/wiki/Mitribah
/wiki/Jangy-Jer
/wiki/Luang_Prabang
None
/wiki/Coloane
/wiki/Chuping
/wiki/Hanimaadhoo_(Haa_Dhaalu_Atoll)
/wiki/Khongor,_Darkhan-Uul
/wiki/Myinmu
/wiki/Dhangadhi
None
/wiki/Mohenjo-daro
/wiki/Tuguegarao
/wiki/Doha
/wiki/Jeddah
/wiki/Tengah,_Singapore
/wiki/Hongcheon
/wiki/Trincomalee
/wiki/Al-Hasakah
/wiki/Fuyuan_National_Forest_Recreation_Area


KeyboardInterrupt: 

In [34]:
df = pd.read_pickle("data/df_with_coordinates_str.pk")

In [37]:
def dms_to_decimal(dms):
    """
    Convert DMS (degrees, minutes, seconds) string to decimal degrees. Minutes and seconds are optional and may include decimals.
    """
    import re
    
    # Match the degrees, optional fractional minutes, and optional fractional seconds with N, S, E, or W
    pattern = re.compile(r'(\d{1,3})°(\d{1,2}(?:\.\d+)?′)?(?:(\d{1,2}(?:\.\d+)?)″)?([NSEW])')
    match = pattern.search(dms)
    
    if not match:
        print(dms)
        raise ValueError("Input does not match DMS format")
    
    degrees, minutes, seconds, direction = match.groups()
    
    # Convert strings to integers or floats
    degrees = int(degrees)
    minutes = float(minutes.rstrip('′')) if minutes else 0.0
    seconds = float(seconds.rstrip('″')) if seconds else 0.0
    
    # Convert to decimal degrees
    decimal = degrees + minutes / 60 + seconds / 3600
    
    # Account for direction South or West
    if direction in 'SW':
        decimal = -decimal
    
    return decimal

In [38]:
df["lat"] = df["coordinates_str"].map(
    lambda lat_long: dms_to_decimal(lat_long[0]) if lat_long[0] is not None else None
)
df["long"] = df["coordinates_str"].map(
    lambda lat_long: dms_to_decimal(lat_long[1]) if lat_long[1] is not None else None
)

In [39]:
df

,temperature_text,color_code,url_suffix,temperature,coordinates_str,lat,long
0,51.3 °C (124.3 °F)\n,#770000,/wiki/Ouargla,51.3,"(31°57′N, 5°19′E)",31.950000,5.316667
1,44.0 °C (111.2 °F)\n,#DC0000,"/wiki/Maun,_Botswana",44.0,"(19°59′S, 023°25′E)",-19.983333,23.416667
2,47.2 °C (117.0 °F)\n,#B00000,"/wiki/Dori,_Burkina_Faso",47.2,"(14°02′N, 0°02′W)",14.033333,-0.033333
3,48.0 °C (118.4 °F)\n,#A50000,/wiki/Faya-Largeau,48.0,"(17°55′01″N, 19°7′0″E)",17.916944,19.116667
4,36.0 °C (96.8 °F)\n,#FF2500,/wiki/Prince_Said_Ibrahim_International_Airport,36.0,"(11°32′12″S, 43°16′17″E)",-11.536667,43.271389
...,...,...,...,...,...,...,...
141,38.0 °C (100.4 °F)\n,#FF1800,/wiki/Saint-Laurent-du-Maroni,38.0,"(5°29′58″N, 54°01′52″W)",5.499444,-54.031111
142,46.2 °C (115.2 °F)\n,#BE0000,/w/index.php?title=Las_Palmas_(Paraguay)&actio...,46.2,"(None, None)",NaN,NaN
143,41.6 °C (106.9 °F)\n,#FD0000,/wiki/I%C3%B1apari,41.6,"(10°56′51.64″S, 69°34′35.04″W)",-10.947678,-69.576400
144,44.0 °C (111.2 °F)\n,#DC0000,/wiki/Paysand%C3%BA,44.0,"(32°19′17″S, 58°4′32″W)",-32.321389,-58.075556


In [36]:
### For asking nicely to ChatGPT : 
print("\n".join(df.coordinates_str.astype(str).tolist()))

('31°57′N', '5°19′E')
('19°59′S', '023°25′E')
('14°02′N', '0°02′W')
('17°55′01″N', '19°7′0″E')
('11°32′12″S', '43°16′17″E')
('24°05′20″N', '32°53′59″E')
('27°19′S', '31°54′E')
(None, None)
('10°53′5″N', '1°5′25″W')
('12°29′N', '13°18′W')
(None, None)
('24°21′S', '44°31′E')
('30°25′17″N', '9°34′59″W')
('13°18′53″N', '12°37′4″E')
('9°14′N', '12°28′E')
('20°56′22″S', '55°17′14″E')
(None, None)
('19°10′11.37″N', '30°28′29.62″E')
('33°42′18″N', '08°57′54″E')
('26°44′22″N', '11°40′13″W')
('12°53′50″S', '31°55′40″E')
('60°42′30″S', '45°35′42″W')
('43°00′N', '41°01′E')
('32°20′37″N', '62°7′10″E')
('24°22′N', '88°36′E')
('26°51′N', '89°23′E')
('13°47′N', '104°58′E')
(None, None)
('22°30′37″N', '114°07′35″E')
('27°07′52″N', '72°21′50″E')
('03°26′33″S', '114°49′57″E')
('31°18′17″N', '48°40′42″E')
('30°30′54″N', '47°48′36″E')
('32°25′18″N', '35°31′28″E')
('36°8′50.6″N', '139°23′19.1″E')
(None, None)
('43°18′07″N', '68°16′09″E')
('29°49′N', '47°22′E')
('40°07′30″N', '70°53′50″E')
('19°53′24″N', '10